In [35]:
# cwd = os.getcwd()
# df_2013 = pd.read_pickle(os.path.join(cwd,'data','df_2013_features.df'))
# df_2014 = pd.read_pickle(os.path.join(cwd,'data','df_2014_features.df'))
import pandas as pd
df_all = pd.read_csv(r"D:\SREE\Project\yield_prediction\data\Crop_yield_cnn.csv")

In [36]:
print(df_all.head())

    Latitude   Longitude  apparentTemperatureMax  apparentTemperatureMin   
0  46.811686 -118.695237                   35.70                   20.85  \
1  46.929839 -118.352109                   35.10                   26.92   
2  47.006888 -118.510160                   33.38                   26.95   
3  47.162342 -118.699677                   28.05                   25.93   
4  47.157512 -118.434056                   28.83                   25.98   

   cloudCover  dewPoint  humidity  precipIntensity  precipIntensityMax   
0        0.00     29.53      0.91           0.0000              0.0000  \
1        0.00     29.77      0.93           0.0001              0.0019   
2        0.00     29.36      0.94           0.0001              0.0022   
3        0.91     29.47      0.94           0.0002              0.0039   
4        0.91     29.86      0.94           0.0003              0.0055   

   precipProbability  ...  precipTypeIsOther  pressure  temperatureMax   
0               0.00  ..

In [37]:
from sklearn.model_selection import train_test_split
y = df_all['yield']
X = df_all.drop('yield',axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [38]:
print(y)

0         35.7
1         35.7
2         35.7
3         35.7
4         35.7
          ... 
360037    50.1
360038    50.1
360039    50.1
360040    50.1
360041    50.1
Name: yield, Length: 360042, dtype: float64


In [39]:
from sklearn import preprocessing
columns_to_scale = df_all.columns.tolist()
columns_to_scale = [x for x in columns_to_scale if x != 'yield']
print(columns_to_scale)

std_scaler = preprocessing.StandardScaler().fit(X_train[columns_to_scale])
minmax_scaler = preprocessing.MinMaxScaler().fit(X_train[columns_to_scale])

X_train[columns_to_scale] = std_scaler.transform(X_train[columns_to_scale])
# X_train[columns_to_scale] = minmax_scaler.transform(X_train[columns_to_scale])

['Latitude', 'Longitude', 'apparentTemperatureMax', 'apparentTemperatureMin', 'cloudCover', 'dewPoint', 'humidity', 'precipIntensity', 'precipIntensityMax', 'precipProbability', 'precipAccumulation', 'precipTypeIsRain', 'precipTypeIsSnow', 'precipTypeIsOther', 'pressure', 'temperatureMax', 'temperatureMin', 'visibility', 'windBearing', 'windSpeed', 'NDVI', 'DayInSeason']


In [40]:
# Don't forget to apply the scaler also to the test set.
X_test[columns_to_scale] = std_scaler.transform(X_test[columns_to_scale])
# X_test[columns_to_scale] = minmax_scaler.transform(X_test[columns_to_scale])


In [41]:
# Use cros validation. Set 'k' and the scoring function. If scoring function is not set, 
# cross_val_score will use the default scoring for the respective estimator. To make sure 
# I compare all estimators on the same scale, I set the scoring function explicitly.
from sklearn.model_selection import cross_val_score
cv_k = 5
cv_scoring = 'neg_mean_squared_error'
cv_scoring = 'r2'

In [42]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=cv_k, shuffle=True)

In [43]:
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn import neighbors
from sklearn.svm import SVR

In [44]:
# Linear regression
now = time.time()
est = linear_model.LinearRegression()
scores = cross_val_score(est, X_train, y_train, cv=kf, scoring=cv_scoring)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
after = time.time()
print('Exec. time: {:5.2f} s'.format(after-now))

NameError: name 'time' is not defined

In [ ]:
# Linear regression with L2 regularization
now = time.time()
est = linear_model.Ridge(alpha = 1.0)
scores = cross_val_score(est, X_train, y_train, cv=kf, scoring=cv_scoring)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
after = time.time()
print('Exec. time: {:5.2f} s'.format(after-now))

In [ ]:
# Linear regression with L2 regularization
now = time.time()
est = linear_model.Ridge(alpha = 1.0)
scores = cross_val_score(est, X_train, y_train, cv=kf, scoring=cv_scoring)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
after = time.time()
print('Exec. time: {:5.2f} s'.format(after-now))

In [ ]:
# Polynomial regression
now = time.time()
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
est = linear_model.LinearRegression()
scores = cross_val_score(est, X_train_poly, y_train, cv=kf, scoring=cv_scoring)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
after = time.time()
print('Exec. time: {:5.2f} s'.format(after-now))

In [ ]:
# Polynomial regression with regularization
now = time.time()
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
est = linear_model.Ridge(alpha = 1.0)
scores = cross_val_score(est, X_train_poly, y_train, cv=kf, scoring=cv_scoring)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
after = time.time()
print('Exec. time: {:5.2f} s'.format(after-now))

In [ ]:
# Random forest regression
now = time.time()
est = RandomForestRegressor(n_estimators=10, n_jobs=-1)
scores = cross_val_score(est, X_train, y_train, cv=kf, scoring=cv_scoring)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
after = time.time()
print('Exec. time: {:5.2f} s'.format(after-now))


In [ ]:
# Gradient boosted regression
now = time.time()
est = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1,
                                max_depth=0.3, random_state=0, loss='ls')
scores = cross_val_score(est, X_train, y_train, cv=kf, scoring=cv_scoring)
print("Accuracy: %0.4f (+/- %0.4f)" % (scores.mean(), scores.std() * 2))
after = time.time()
print('Exec. time: {:5.2f} s'.format(after-now))